In [22]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pickle
import time
import random
from convnet import *
from remtime import *

In [23]:
## Hyperparameters
NUM_OUTPUT = 10
LEARNING_RATE = 0.01	#learning rate
IMG_WIDTH = 28
IMG_DEPTH = 1
FILTER_SIZE=5
NUM_FILT1 = 8
NUM_FILT2 = 8
BATCH_SIZE = 1
NUM_EPOCHS = 2	 # number of iterations
MU = 0.95

PICKLE_FILE = 'output.pickle'
# PICKLE_FILE = 'trained.pickle'

In [24]:
## Data extracting
m =10000
X = extract_data('t10k-images-idx3-ubyte.gz', m, IMG_WIDTH)
y_dash = extract_labels('t10k-labels-idx1-ubyte.gz', m).reshape(m,1)
X-= int(np.mean(X))
X/= int(np.std(X))
test_data = np.hstack((X,y_dash))


m =50000
X = extract_data('train-images-idx3-ubyte.gz', m, IMG_WIDTH)
y_dash = extract_labels('train-labels-idx1-ubyte.gz', m).reshape(m,1)
print np.mean(X), np.std(X)
X-= int(np.mean(X))
X/= int(np.std(X))
train_data = np.hstack((X,y_dash))

('Extracting', 't10k-images-idx3-ubyte.gz')
('Extracting', 't10k-labels-idx1-ubyte.gz')
('Extracting', 'train-images-idx3-ubyte.gz')
('Extracting', 'train-labels-idx1-ubyte.gz')
33.395157 78.66619


In [25]:
np.random.seed(0)
np.random.shuffle(train_data)

In [26]:
NUM_IMAGES = train_data.shape[0]

## Initializing all the parameters
filt1 = {}
filt2 = {}
bias1 = {}
bias2 = {}
for i in range(0,NUM_FILT1):
	filt1[i] = initialise_param_lecun_normal(FILTER_SIZE, IMG_DEPTH, scale=1.0, distribution='normal')
	bias1[i] = 0
	# v1[i] = 0
for i in range(0,NUM_FILT2):
	filt2[i] = initialise_param_lecun_normal(FILTER_SIZE, NUM_FILT1, scale=1.0, distribution='normal')
	bias2[i] = 0
	# v2[i] = 0
w1 = IMG_WIDTH-FILTER_SIZE+1
w2 = w1-FILTER_SIZE+1
theta3 = initialize_theta(NUM_OUTPUT, (w2/2)*(w2/2)*NUM_FILT2)

bias3 = np.zeros((NUM_OUTPUT,1))
cost = []
acc = []
# pickle_in = open(PICKLE_FILE, 'rb')
# out = pickle.load(pickle_in)

# [filt1, filt2, bias1, bias2, theta3, bias3, cost, acc] = out


print("Learning Rate:"+str(LEARNING_RATE)+", Batch Size:"+str(BATCH_SIZE))

Learning Rate:0.01, Batch Size:1


In [19]:
for epoch in range(0,NUM_EPOCHS):
	batches = [train_data[k:k + BATCH_SIZE] for k in xrange(0, NUM_IMAGES, BATCH_SIZE)]
	x=0
	for batch in batches:
		stime = time.time()
		print(len(batch))
		# LEARNING_RATE =  LEARNING_RATE/(1+epoch/10.0)
		out = momentumGradDescent(batch, LEARNING_RATE, IMG_WIDTH, IMG_DEPTH, MU, filt1, filt2, bias1, bias2, theta3, bias3, cost, acc, order = 'n',path = 'b')
		[filt1, filt2, bias1, bias2, theta3, bias3, cost, acc] = out
		epoch_acc = round(np.sum(acc[epoch*NUM_IMAGES/BATCH_SIZE:])/(x+1),2)
		
		per = float(x+1)/len(batches)*100
		print("Epoch:"+str(round(per,2))+"% Of "+str(epoch+1)+"/"+str(NUM_EPOCHS)+", Cost:"+str(cost[-1])+", B.Acc:"+str(acc[-1]*100)+", E.Acc:"+str(epoch_acc))
		
		ftime = time.time()
		deltime = ftime-stime
		remtime = (len(batches)-x-1)*deltime+deltime*len(batches)*(NUM_EPOCHS-epoch-1)
		printTime(remtime)
		x+=1
		break        

1
Epoch:0.0% Of 1/2, Cost:[2.23286043], B.Acc:0.0, E.Acc:0.0
########  3Hrs 41Mins 4Secs remaining  ########
1
Epoch:0.0% Of 2/2, Cost:[0.], B.Acc:100.0, E.Acc:0.0
########  1Hrs 51Mins 31Secs remaining  ########


In [21]:
filt1

{0: array([[[ 0.35301127,  0.08017187,  0.19583711,  0.44830023,
           0.37349178],
         [-0.19552592,  0.18982603, -0.03029294, -0.0205884 ,
           0.08205764],
         [ 0.02864477,  0.29076107,  0.152389  ,  0.02439906,
           0.08850514],
         [ 0.06644242,  0.29879034, -0.04087093,  0.06294114,
          -0.17061162],
         [-0.51081215,  0.13071095,  0.17323002, -0.14799804,
           0.45423048]]]),
 1: array([[[ 0.35306768,  0.0802273 ,  0.19577066,  0.44827473,
           0.37364636],
         [-0.19551085,  0.1899598 , -0.03026293, -0.02068815,
           0.08208163],
         [ 0.02877793,  0.29080847,  0.15219261,  0.02398728,
           0.08832142],
         [ 0.06680108,  0.29899763, -0.04124108,  0.06228763,
          -0.17123438],
         [-0.51034343,  0.13072335,  0.17269224, -0.14877439,
           0.45357577]]]),
 2: array([[[ 0.35283046,  0.08019977,  0.19593914,  0.44824944,
           0.37350892],
         [-0.19533533,  0.19018372, -0.

In [20]:
filt1

{0: array([[[ 0.35301127,  0.08017187,  0.19583711,  0.44830023,
           0.37349178],
         [-0.19552592,  0.18982603, -0.03029294, -0.0205884 ,
           0.08205764],
         [ 0.02864477,  0.29076107,  0.152389  ,  0.02439906,
           0.08850514],
         [ 0.06644242,  0.29879034, -0.04087093,  0.06294114,
          -0.17061162],
         [-0.51081215,  0.13071095,  0.17323002, -0.14799804,
           0.45423048]]]),
 1: array([[[ 0.35306768,  0.0802273 ,  0.19577066,  0.44827473,
           0.37364636],
         [-0.19551085,  0.1899598 , -0.03026293, -0.02068815,
           0.08208163],
         [ 0.02877793,  0.29080847,  0.15219261,  0.02398728,
           0.08832142],
         [ 0.06680108,  0.29899763, -0.04124108,  0.06228763,
          -0.17123438],
         [-0.51034343,  0.13072335,  0.17269224, -0.14877439,
           0.45357577]]]),
 2: array([[[ 0.35283046,  0.08019977,  0.19593914,  0.44824944,
           0.37350892],
         [-0.19533533,  0.19018372, -0.

In [27]:
for epoch in range(0,NUM_EPOCHS):
	batches = [train_data[k:k + BATCH_SIZE] for k in xrange(0, NUM_IMAGES, BATCH_SIZE)]
	x=0
	for batch in batches:
		stime = time.time()
		# LEARNING_RATE =  LEARNING_RATE/(1+epoch/10.0)
		out = momentumGradDescent(batch, LEARNING_RATE, IMG_WIDTH, IMG_DEPTH, MU, filt1, filt2, bias1, bias2, theta3, bias3, cost, acc, order = 'a',path = 'b')
		[filt1, filt2, bias1, bias2, theta3, bias3, cost, acc] = out
		epoch_acc = round(np.sum(acc[epoch*NUM_IMAGES/BATCH_SIZE:])/(x+1),2)
		
		per = float(x+1)/len(batches)*100
		print("Epoch:"+str(round(per,2))+"% Of "+str(epoch+1)+"/"+str(NUM_EPOCHS)+", Cost:"+str(cost[-1])+", B.Acc:"+str(acc[-1]*100)+", E.Acc:"+str(epoch_acc))
		
		ftime = time.time()
		deltime = ftime-stime
		remtime = (len(batches)-x-1)*deltime+deltime*len(batches)*(NUM_EPOCHS-epoch-1)
		printTime(remtime)
		x+=1
		break

Epoch:0.0% Of 1/2, Cost:[2.23286043], B.Acc:0.0, E.Acc:0.0
########  5Hrs 13Mins 46Secs remaining  ########
Epoch:0.0% Of 2/2, Cost:[0.], B.Acc:100.0, E.Acc:0.0
########  2Hrs 34Mins 31Secs remaining  ########


In [28]:
filt1

{0: array([[[ 0.35301127,  0.08017187,  0.19583711,  0.44830023,
           0.37349178],
         [-0.19552592,  0.18982603, -0.03029294, -0.0205884 ,
           0.08205764],
         [ 0.02864477,  0.29076107,  0.152389  ,  0.02439906,
           0.08850514],
         [ 0.06644242,  0.29879034, -0.04087093,  0.06294114,
          -0.17061162],
         [-0.51081215,  0.13071095,  0.17323002, -0.14799804,
           0.45423048]]]),
 1: array([[[ 0.35306768,  0.0802273 ,  0.19577066,  0.44827473,
           0.37364636],
         [-0.19551085,  0.1899598 , -0.03026293, -0.02068815,
           0.08208163],
         [ 0.02877793,  0.29080847,  0.15219261,  0.02398728,
           0.08832142],
         [ 0.06680108,  0.29899763, -0.04124108,  0.06228763,
          -0.17123438],
         [-0.51034343,  0.13072335,  0.17269224, -0.14877439,
           0.45357577]]]),
 2: array([[[ 0.35283046,  0.08019977,  0.19593914,  0.44824944,
           0.37350892],
         [-0.19533533,  0.19018372, -0.

In [ ]:
A = np.array([[[1, 3, 2, 7,1],
           [2, 4, 1, 3,1],
           [6, 1, 2, 3,1],[6, 1, 2, 3,1],[2, 4, 7, 4,1]],[[1, 3, 2, 7,1],
           [2, 4, 1, 3,1],
           [6, 1, 2, 3,1],[6, 1, 2, 3,1],[2, 4, 7, 4,1]]])

In [ ]:
A.shape

In [ ]:
B = np.array([[[2, 2, 1, 5,1],
           [1, 3, 1, 2,1],
           [2, 4, 7, 4,1],
           [2, 4, 7, 4,1],[2, 4, 7, 4,1]],[[2, 2, 1, 5,1],
           [1, 3, 1, 2,1],
           [2, 4, 7, 4,1],
           [2, 4, 7, 4,1],[2, 4, 7, 4,1]]])

In [ ]:
B.shape

In [ ]:
p = []

In [ ]:
p.append(A)

In [ ]:
p.append(B)

In [ ]:
p[0].shape

In [ ]:
p

In [ ]:
np.stack(p, axis = 1).shape

In [ ]:
np.stack(p, axis = 1)[0].shape

In [ ]:
np.stack(np.stack(p, axis = 1)[0], axis = -1)

In [ ]:
np.stack(np.stack(p, axis = 1)[0], axis = -1).shape

In [ ]:
np.sort(np.stack(np.stack(p, axis = 1)[0], axis=-1),axis=2)

In [ ]:
np.sum(np.sort(np.stack(np.stack(p, axis = 1)[0], axis=-1),axis=2),axis=2)

In [ ]:
np.sort(np.stack(np.stack(p, axis = 1)[0], axis=-1),axis=2)[::-1]

In [ ]:
np.stack(np.stack(p, axis = 1)[0], axis=-1)

In [ ]:
-np.sort(-np.stack(np.stack(p, axis = 1)[0], axis=-1),axis=2)

In [ ]:
np.sum(np.sort(np.stack(np.stack(p, axis = 1)[0], axis=-1),axis=2)[::-1],axis=2)

In [ ]:
G[0]

In [ ]:
G = A + B

In [ ]:
np.sum(np.sort(np.sum(np.sort(np.stack(C, axis=-1),axis=2),axis=2).flatten()))